In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

In [2]:
patch_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/'

In [25]:
# stack = 'MD594'
stack = 'MD589'
dm = DataManager(stack=stack, load_mask=False)

In [26]:
#define roi1
if stack == 'MD594':
    first = 157
    last = 166

    box_x = 99
    box_y = 140
    box_w = 347
    box_h = 190
    
elif stack == 'MD589':
    first = 154
    last = 168

    box_x = 38
    box_y = 137
    box_w = 355
    box_h = 221
    
xmin = max(half_size, box_x*32)
xmax = min(dm.image_width-half_size-1, (box_x+box_w)*32)
ymin = max(half_size, box_y*32)
ymax = min(dm.image_height-half_size-1, (box_y+box_h)*32)

In [27]:
patch_size = 224
half_size = patch_size/2
stride = 56

ys, xs = np.meshgrid(np.arange(half_size, dm.image_height - half_size, stride), 
                     np.arange(half_size, dm.image_width - half_size, stride),
                     indexing='xy')

sample_locations = np.c_[xs.flat, ys.flat]

In [28]:
indices_roi_allSections = defaultdict(dict)

for sec in range(first, last+1):

    dm.set_slice(sec)

    mask = dm.load_thumbnail_mask()
    indices_fg = np.where(mask[sample_locations[:,1]/32, sample_locations[:,0]/32])[0]
    indices_bg = np.setdiff1d(range(sample_locations.shape[0]), indices_fg)

    indices_roi = np.where(np.all(np.c_[sample_locations[:,0] > xmin, sample_locations[:,1] > ymin, 
                                        sample_locations[:,0] < xmax, sample_locations[:,1] < ymax], axis=1))[0]

    indices_roi = np.setdiff1d(indices_roi, indices_bg)
    print len(indices_roi), 'patches in ROI'
    
    indices_roi_allSections[sec]['roi1'] = indices_roi

20019 patches in ROI
20218 patches in ROI
20623 patches in ROI
20755 patches in ROI
20898 patches in ROI
20831 patches in ROI
20877 patches in ROI
20904 patches in ROI
21049 patches in ROI
21289 patches in ROI
21248 patches in ROI
21477 patches in ROI
21502 patches in ROI
21543 patches in ROI
21793 patches in ROI


In [29]:
import pandas as pd

In [30]:
indices_roi_allSections_df = pd.DataFrame(indices_roi_allSections)

In [31]:
indices_roi_allSections_df

,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168
roi1,"[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420...","[4197, 4198, 4199, 4200, 4201, 4202, 4203, 420..."


In [32]:
dump_path = os.path.join(patch_dir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})

In [33]:
indices_roi_allSections_df.to_hdf(dump_path, 'indices_allROIs_allSections')
pd.Series([patch_size, stride, dm.image_width, dm.image_height]).to_hdf(dump_path, 'grid_parameters')

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->[154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168]]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
